In [1]:
#imports
import tensorflow as tf 
import numpy as np
import random

In [9]:
#constants
data_points = 12800

# test_points = 1000
test_point = np.array([[[10], [1], [3], [0], [4], [4], [2], [5]]])

n_input = 1
max_seq_len = 20
max_int = 100
n_hidden = 30
n_outputs = 1
batch_size = 128
lr = 0.001
training_steps = 50000

In [10]:
#data
"""
X[i] has shape (data_points(turns into batch_size), seq_length=i, 
                                                    num_features(1 in this case since it is a single number))
Y[i] has shape (data_points(turns into batch_size), num_features) for each i
"""
def generate_training_data(data_points, seq_length):
    x_data = []
    y_data = []
    for _ in range(data_points):
        x_point = [[random.randint(1, max_int)] for e in range(seq_length)]
        x_data.append(x_point)
        y_data.append([sum([i[0] for i in x_point])])
    return np.array(x_data), np.array(y_data)

def gen_data(max_len):
    batches_x = []
    batches_y = []
    for length in range(1, max_len+1):
        batch_x, batch_y = generate_training_data(data_points, length)
        batches_x.append(batch_x)
        batches_y.append(batch_y)
    return batches_x, batches_y

X, Y = gen_data(max_seq_len)

In [11]:
#model
with tf.variable_scope("scope_1", reuse=tf.AUTO_REUSE):
    weights = tf.Variable(tf.random.normal([n_hidden, n_outputs]))
    biases = tf.Variable(tf.random.normal([n_outputs]))
 
    x = tf.placeholder(tf.float32, [None, None, n_input])
    y = tf.placeholder(tf.float32, [None, 1])

    cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
    
    final_output = tf.add(tf.matmul(outputs[:,-1], weights), biases)
        
    mse = tf.losses.mean_squared_error(y, final_output)
    loss = tf.reduce_mean(mse)
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    training_op = optimizer.minimize(loss)

In [12]:
def get_batch_helper(x, y, batch_size):
    i = random.randint(0, data_points-batch_size)
    return x[i:i+batch_size], y[i:i+batch_size]

def get_batch(x, y, batch_size):
    i = random.randint(0, max_seq_len-1)
    return get_batch_helper(x[i], y[i], batch_size)

In [13]:
#run computation graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(training_steps):
        batch_x, batch_y = get_batch(X, Y, batch_size)
        _, curr_loss = sess.run([training_op, loss], feed_dict={x: batch_x, y: batch_y})
        if step%1000 == 0:
            pred = sess.run([final_output], feed_dict={x: test_point})
            print("Loss:", curr_loss)
            print("Pred:", pred)

Loss: 639368.6
Pred: [array([[3.3239763]], dtype=float32)]
Loss: 102643.62
Pred: [array([[48.971855]], dtype=float32)]
Loss: 292483.12
Pred: [array([[73.940895]], dtype=float32)]
Loss: 97298.09
Pred: [array([[16.045238]], dtype=float32)]
Loss: 422144.75
Pred: [array([[12.557551]], dtype=float32)]
Loss: 1829.1031
Pred: [array([[40.10903]], dtype=float32)]
Loss: 41585.23
Pred: [array([[49.525276]], dtype=float32)]
Loss: 463.90936
Pred: [array([[27.851309]], dtype=float32)]
Loss: 497511.0
Pred: [array([[25.285046]], dtype=float32)]
Loss: 430528.25
Pred: [array([[29.88387]], dtype=float32)]
Loss: 27056.754
Pred: [array([[29.128147]], dtype=float32)]
Loss: 76.101456
Pred: [array([[29.807024]], dtype=float32)]
Loss: 334087.0
Pred: [array([[28.458363]], dtype=float32)]
Loss: 11980.556
Pred: [array([[29.145374]], dtype=float32)]
Loss: 136793.75
Pred: [array([[29.577305]], dtype=float32)]
Loss: 215440.1
Pred: [array([[21.60611]], dtype=float32)]
Loss: 175.08109
Pred: [array([[25.062105]], dtype